In [1]:
import requests as req

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [4]:
import sqlite3 as sql

In [5]:
import requests

In [6]:
from functools import reduce

In [7]:
sqlite_file = 'salaries_first_last.sql'

In [8]:
db = sql.connect(":memory:")

In [9]:
sql_file = open(sqlite_file)
sql_as_string = sql_file. read()
db. executescript(sql_as_string)

In [27]:
salary_query= """
SELECT 
 firstname, lastname, department,  salary
FROM Salaries
"""

In [28]:
salaries = pd.read_sql(salary_query, db)

In [29]:
salaries

,firstname,lastname,department,salary
0,DJ,Durkin,PRES,881010.96
1,Darryll J,Pines,PRES,765000.00
2,Wallace,Loh,PRES,734565.88
3,Mark,Turgeon,PRES,723312.99
4,Brenda S,Frese,PRES,659386.43
...,...,...,...,...
10125,Benjamin Leonard,Worsley,JOUR,8241.60
10126,Joseph,Yasharoff,JOUR,8241.60
10127,Denitsa Haralampieva,Yotova,JOUR,8241.60
10128,Jerry,Zremski,JOUR,8241.60


In [37]:
headers = {
  'Accept': 'application/json'
}

profs = []

offset = 0;
cont = True
for i in range(0, 2000, 1000):
    r = requests.get('https://api.planetterp.com/v1/professors', params={
      'limit': '1000',
      'offset': i
    }, headers = headers)

    ret = r.json();
    
    profs = profs + ret ;
    
print(len(profs))

2000


In [38]:
def find_closest_match(name, matches):
    firstname = name.split(" ")[0]
    ret = [ m  for m in matches if m[0].find(firstname) >= 0 ]
    if(len(ret) > 0):
        return ret[0]
    return None

In [39]:
def get_department(dep):
    matches = dict()

    for prof in profs:
        slug = prof.get("slug")
        name = prof.get("name")
        lastname = name.split(" ")[-1]
        salary_query= f"""
            SELECT 
            firstname, 
            lastname,
            department,
            salary
            FROM Salaries
            WHERE lastname like "{lastname}"
            AND department = "{dep}"
        """
        rows = list(db.execute(salary_query))
        matches[lastname] = (len(rows), rows, name)

    nonzero = []
    total = 0
    for match in matches.keys():
        (length, v, name) = matches.get(match)
        if(length > 0):
        
            value = find_closest_match(name, v)
            if(value != None):
#                 print("the value ", value)
                nonzero.append((value, name))
                total += 1
            
    return nonzero


In [40]:
arr = list(set(salaries["department"]))
matches = reduce(lambda a, c:  a + c  , map(lambda x: get_department(x), arr), [])
matches

[(('Ana', 'Palla-Kane', 'DIT', 130528.77), 'Ana Palla-Kane'),
 (('Bertrand', 'Sobesto', 'DIT', 104983.3), 'Bertrand Sobesto'),
 (('Adrienne Hamcke', 'Wicker', 'SVPAAP', 107237.29), 'Adrienne Wicker'),
 (('Alice Elizabeth', 'Donlan', 'SVPAAP', 87019.18), 'Alice Donlan'),
 (('Cherisse La-Jae', 'Hall', 'SVPAAP', 66300.0), 'Cherisse Hall'),
 (('Brittany K', 'Lashley', 'SVPAAP', 52212.6), 'Brittany Lashley'),
 (('Benjamin David', 'Huffman', 'SVPAAP', 73950.0), 'Benjamin Huffman'),
 (('Bonnie', 'Miranda', 'SVPAAP', 76141.79), 'Bonnie Miranda'),
 (('Brian', 'Pierce', 'SVPAAP', 124485.29), 'Brian Pierce'),
 (('Carly E', 'Jimeson', 'SVPAAP', 66660.0), 'Carly Jimeson'),
 (('Carolina', 'Vieira', 'SVPAAP', 62001.17), 'Carolina Vieira'),
 (('Cate OBrien', 'Barger', 'SVPAAP', 71790.82), 'Cate Barger'),
 (('Catherine', 'Spirito', 'SVPAAP', 67439.87), 'Catherine Spirito'),
 (('Christina Louise', 'Hnatov', 'SVPAAP', 38632.5), 'Christina Hnatov'),
 (('Conrad', 'Zeutenhorst', 'SVPAAP', 54109.16), 'Conrad

In [41]:
len(matches)   

484

In [36]:
headers = {
  'Accept': 'application/json'
}

data = []
for index, row in salaries.iterrows():
    firstname = row["firstname"]
    lastname = row["lastname"]
    r = requests.get(f'https://api.umd.io/v1/professors?name={firstname} {lastname}', params={
      'limit': '1000'
    }, headers = headers)
    data.append(r.json())
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data